In [10]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

In [124]:
from tqdm import tqdm
def load_all_adf1():
    dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CrWSe2_total 8 datasets\CrWSe2_20211028'+'\\'

    nums = [6, 7, 9, 13, 14, 15, 17]

    imgs1 = []
    for num in tqdm(nums):
        # load images
        file_name1 = dp + 'ADF1_{}.dm4'.format(num)
        file_name2 = dp + 'ADF2_{}.dm4'.format(num)
        
        # load image and normalize
        img1 = load_image(file_name1)
        img1 = normalize_image(img1, 0, 1)

        imgs1.append(img1)
    return np.array(imgs1)

def load_all_adf2():
    dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CrWSe2_total 8 datasets\CrWSe2_20211028'+'\\'

    nums = [6, 7, 9, 13, 14, 15, 17]

    imgs1 = []
    for num in tqdm(nums):
        # load images
        file_name1 = dp + 'ADF1_{}.dm4'.format(num)
        file_name2 = dp + 'ADF2_{}.dm4'.format(num)
        
        # load image and normalize
        img1 = load_image(file_name2)
        img1 = normalize_image(img1, 0, 1)

        imgs1.append(img1)
    return np.array(imgs1)

In [123]:
imgs1 = load_all_adf1()
np.save('all_adf1.np', imgs1)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 23.89it/s]


In [125]:
imgs2 = load_all_adf2()
np.save('all_adf2.np', imgs2)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.28it/s]


In [126]:
imshow(imgs1)

## stop here

In [115]:
from skimage.transform import warp_polar
from scipy.ndimage import uniform_filter1d

def baseline_correction(y,niter=10):
    n = len(y)
    y_ = np.log(np.log(np.sqrt(y +1)+1)+1)
    yy = np.zeros_like(y)

    for pp in np.arange(1,niter+1):
        r1 = y_[pp:n-pp]
        r2 = (np.roll(y_,-pp)[pp:n-pp] + np.roll(y_,pp)[pp:n-pp])/2
        yy = np.minimum(r1,r2)
        y_[pp:n-pp] = yy

    baseline = (np.exp(np.exp(y_)-1)-1)**2 -1
    return baseline

def get_fft_line(img, niter=10, size=9, use_log=True, debug=True):
    fft_abs = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    if use_log:
        fft_log = np.log(fft_abs + 1)
    else:
        fft_log = fft_abs
    i ,j = np.unravel_index(np.argmax(fft_log), shape=img.shape)
    y = warp_polar(fft_log, center=(i, j)).mean(axis=0)[0:i]
    bg = baseline_correction(y, niter=niter)
    y1 = y - bg
    y2 = uniform_filter1d(y1,size=size)
    return y2

class TMDImage:

    def __init__(self, img):
        self.img = img
        self.fft_line = get_fft_line(self.img, debug=False)
        ind = np.argmax(self.fft_line)
        size = np.round(self.img.shape[0]/ind).astype(int)
        self.patch_size = 2*size + 1

    

In [34]:
from tqdm import tqdm
def load_all_adf1():
    dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CrWSe2_total 8 datasets\CrWSe2_20211028'+'\\'

    nums = [6, 7, 9, 13, 14, 15, 17]

    lines = []
    for num in tqdm(nums):
        # load images
        file_name1 = dp + 'ADF1_{}.dm4'.format(num)
        file_name2 = dp + 'ADF2_{}.dm4'.format(num)
        
        # load image and normalize
        img1 = load_image(file_name1)
        img1 = normalize_image(img1, 0, 1)

        img1 = remove_bg(img1, 7)

        line = get_fft_line(img1)
        lines.append(line)

        print(2*get_patch_size(img1, debug=False)+3)
    return lines.

In [35]:
lines = load_all_adf1()

 14%|████████████                                                                        | 1/7 [00:00<00:01,  3.42it/s]

21


 29%|████████████████████████                                                            | 2/7 [00:00<00:01,  3.43it/s]

19


 43%|████████████████████████████████████                                                | 3/7 [00:00<00:01,  3.41it/s]

...

17


 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:01<00:00,  3.44it/s]

21


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.41it/s]

21


In [75]:
for ii, line in enumerate(lines):
    plt.plot(line+ii*0.3)

In [49]:
def get_feature_X(num=6, threshold = 0.3, min_distance=1):
    dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CrWSe2_total 8 datasets\CrWSe2_20211028'+'\\'

    # load images
    file_name1 = dp + 'ADF1_{}.dm4'.format(num)
    file_name2 = dp + 'ADF2_{}.dm4'.format(num)
    
    # load image and normalize
    img1 = load_image(file_name1)
    img1 = normalize_image(img1, 0, 1)
    
    img2 = load_image(file_name2)
    img2 = normalize_image(img2, 0, 1)

    # clean images
    img1 = remove_bg(img1, 7)
    img2 = remove_bg(img2, 7)
    
    img1 = normalize_image(img1, 0, 1)
    img2 = normalize_image(img2, 0, 1)

    # denoise img2
    n_components = 32
    patch_size = 32
    extraction_step = 2
    imgf = denoise_svd(img2, n_components, patch_size, extraction_step, verbose=False) 

    # find points
    pts = local_max(imgf, min_distance=min_distance, threshold=threshold, plot=False)

    size = 2*get_patch_size(img1, debug=False)+3
    kp1 = KeyPoints(pts, img1, size)
    ps1 = kp1.extract_patches(size) 
    
    kp2 = KeyPoints(pts, img2, size)
    ps2 = kp2.extract_patches(size)

    zps = ZPs(n_max=10, size=ps1.shape[1])
    zps.fit(ps1)
    X1 = zps.moments
    zps.fit(ps2)
    X2 = zps.moments
    X = X1.hstack(X2)
    return X

In [78]:
def vstack_zmarrays(a):
    a0 = a[0]
    for e in a[1:]:
        a0 = a0.vstack(e)
    return a0

In [52]:
nums = [6, 7, 9, 13, 14, 15, 17]
thresholds = [0.3, 0.3, 0.3, 0.1, 0.1, 0.1, 0.1]
min_dists = [1, 1, 3, 1, 1, 1, 1]

In [67]:
ll = []
lbs_ = []
ii = 0
for num, threshold, min_distance in tqdm(zip(nums, thresholds, min_dists ), total=7):
    X = get_feature_X(num, threshold, min_distance)
    ll.append(X)
    lbs_.append([ii]*len(X))
    ii = ii + 1
lbs_ = np.hstack(lbs_)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:25<00:00,  3.69s/it]


In [83]:
X = vstack_zmarrays(ll)

In [84]:
lbs = gmm_lbs(X, 2, n_init=1)

In [88]:
X1 = X[lbs==0]
X2 = X[lbs==1]
lbs1 = lbs_[lbs==0]
lbs2 = lbs_[lbs==1]

In [111]:
#X11 = np.hstack([X1, X1.rotinv().select(0)])
#X22 = np.hstack([X2, X2.rotinv().select(0)])

In [112]:
plot_pca(X1.rotinv().select(0), s=1, lbs=lbs1)

In [113]:
plot_pca(X2.rotinv().select(0), s=1, lbs=lbs2)